In [1]:
!git clone https://github.com/e9t/nsmc.git
!pip install tensorflow_addons
!pip install transformers

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1
Receiving objects: 100% (14763/14763), 56.19 MiB | 17.50 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from transformers import pipeline, AutoTokenizer, BertTokenizer, BertTokenizerFast
from transformers import AutoModel, TFBertModel, TFBertForSequenceClassification

In [3]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

TPU = True
if TPU:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# !git clone https://github.com/e9t/nsmc.git
train = pd.read_table("nsmc/"+"ratings_train.txt")
train = train.dropna().sample(5000).reset_index().iloc[:,1:]
test = pd.read_table("nsmc/"+"ratings_test.txt")
test = test.dropna().sample(5000).reset_index().iloc[:,1:]

def preprocessing_sentence_to_BERTinput(df, tokenizer, colname_data, colname_target=None, seq_len=128,
                                        return_type='tensor'):
    tokens, masks, segments, targets = [], [], [], []
    for i in tqdm(range(len(df))):
        # 변환
        token = tokenizer.encode_plus(df[colname_data][i], max_length=seq_len,
                                      pad_to_max_length=True, truncation=True,
                                      return_attention_mask=True,
                                      add_special_tokens=True)

        # 정리
        tokens.append(token['input_ids'])
        masks.append(token['attention_mask'])
        segments.append(token['token_type_ids'])
        if colname_target != None:
            targets.append(df[colname_target][i])

    # array 변환
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    if colname_target != None:
        targets = np.array(targets)

    # tensor 변환
    if return_type == 'tensor':
        tokens = tf.convert_to_tensor(tokens, dtype=tf.int32)
        masks = tf.convert_to_tensor(masks, dtype=tf.int32)
        segments = tf.convert_to_tensor(segments, dtype=tf.int32)

    return [tokens, masks, segments], targets

import tensorflow_addons as tfa
from transformers import pipeline, AutoTokenizer, BertTokenizer, BertTokenizerFast
from transformers import AutoModel, AutoModelForTokenClassification, TFBertModel, TFBertForSequenceClassification

MODEL_NAME = 'monologg/kobert'    # 'bert-base-multilingual-cased'
# OPTIMIZER = tfa.optimizers.RectifiedAdam(lr=1.0e-5, weight_decay=0.0025, warmup_proportion=0.05)
OPTIMIZER = tf.keras.optimizers.Adam(lr=1.0e-5)
NUM_LABELS = 2
SEQ_LEN = 64

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
X_train, Y_train = preprocessing_sentence_to_BERTinput(train, tokenizer=tokenizer,
                                                       colname_target='label', colname_data='document', seq_len=SEQ_LEN)
X_test, Y_test = preprocessing_sentence_to_BERTinput(test, tokenizer=tokenizer,
                                                       colname_target='label', colname_data='document', seq_len=SEQ_LEN)

def modeling_BERTsentiment(model_name, optimizer, num_labels=2, seq_len=128):
    # 모델 로딩
    model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=metric)

    return model

# def modeling_BERTsentiment(model_name, optimizer, num_labels=2, seq_len=128):
#     # 모델 로딩
#     model = TFBertModel.from_pretrained(model_name, num_labels=num_labels, output_hidden_states=True)
#     outputs = model([tokens, masks, segments])[1]

#     # 모델 구성
#     layer = tf.keras.layers.Dense(1, activation='sigmoid',
#                                   kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(outputs)
#     model_sentiment = tf.keras.Model([tokens, masks, segments], layer)
#     model_sentiment.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

#     return model_sentiment


model = modeling_BERTsentiment(model_name=MODEL_NAME, optimizer=OPTIMIZER, num_labels=NUM_LABELS, seq_len=SEQ_LEN)
model.fit(X_train, Y_train, epochs=10, shuffle=True, batch_size=100, validation_data=(X_test, Y_test))


  0%|          | 0/5000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 5000/5000 [00:02<00:00, 1761.60it/s]
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and in

Epoch 1/10
50/50 [==============================] - 323s 6s/step - loss: 0.7157 - accuracy: 0.5114 - val_loss: 0.6977 - val_accuracy: 0.4874
Epoch 2/10
50/50 [==============================] - 293s 6s/step - loss: 0.7056 - accuracy: 0.5054 - val_loss: 0.7149 - val_accuracy: 0.5126
Epoch 3/10
50/50 [==============================] - 294s 6s/step - loss: 0.6976 - accuracy: 0.5128 - val_loss: 0.6957 - val_accuracy: 0.5126
Epoch 4/10
50/50 [==============================] - 294s 6s/step - loss: 0.6981 - accuracy: 0.5008 - val_loss: 0.6933 - val_accuracy: 0.4874
Epoch 5/10
50/50 [==============================] - 294s 6s/step - loss: 0.7017 - accuracy: 0.4920 - val_loss: 0.6941 - val_accuracy: 0.5126
Epoch 6/10
50/50 [==============================] - 295s 6s/step - loss: 0.7003 - accuracy: 0.4952 - val_loss: 0.6928 - val_accuracy: 0.5126
Epoch 7/10
50/50 [==============================] - 294s 6s/step - loss: 0.6954 - accuracy: 0.5056 - val_loss: 0.6954 - val_accuracy: 0.5126
Epoch 8/10
50